In [2]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
import torch
from transformers import Trainer,TrainingArguments
from transformers import BertTokenizer,BertForSequenceClassification
np.set_printoptions(suppress=True)
import tensorflow as tf

In [3]:
extreme_left=pd.read_csv("/Users/dhanushkikkisetti/Documents/Research Assistant/ExtremeLeft.csv")
extreme_right=pd.read_csv("/Users/dhanushkikkisetti/Documents/Research Assistant/ExtremeRight.csv") 
extreme_left['label']=0
extreme_right['label']=1
extreme_left=extreme_left.loc[:,['text','label']]
extreme_right=extreme_right.loc[:,['text','label']]
extreme_right=extreme_right.sample(2014)
left_right=pd.concat([extreme_left,extreme_right])
text_data=left_right['text'].to_list()
label_data=left_right['label'].to_list()

In [5]:
x_train,x_val,y_train,y_val=train_test_split(text_data,label_data,test_size=0.2,stratify=label_data)
#For testing the model
#x_train,x_test,y_train,y_test=train_test_split(text_data,label_data,test_size=0.02,stratify=label_data)

In [14]:
tokenizer=BertTokenizer.from_pretrained("bert-base-uncased")
model=BertForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [7]:
x_train_tokenize=tokenizer(x_train,padding=True,truncation=True,max_length=512)
x_val_tokenize=tokenizer(x_val,padding=True,truncation=True,max_length=512)
x_train_tokenize.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

## Method1 for fine tuning

In [8]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)



In [9]:
X_train_dataset=Dataset(x_train_tokenize,y_train)
X_val_dataset=Dataset(x_val_tokenize,y_val)

In [10]:
def compute_metrics(p):
    print(type(p))
    predected_label,label=p
    predected_label=np.argmax(predected_label,axis=1)
    
    accuracy=accuracy_score(y_pred=predected_label,y_true=label)
    recall=recall_score(y_pred=predected_label,y_true=label)
    precission=precision_score(y_pred=predected_label,y_true=label)
    f1score=f1_score(y_pred=predected_label,y_true=label)
    
    return {"Accuracy Score ":accuracy,"Recall Score ":recall,"Precission Score" :precission,"F1 Score ":f1score} 

In [16]:
args=TrainingArguments(
    output_dir="output",
    num_train_epochs=5,
    per_device_train_batch_size=8
)

trainer=Trainer(
    model=model,
    args=args,
    train_dataset=X_train_dataset,
    eval_dataset=X_val_dataset,
    compute_metrics=compute_metrics
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [17]:
trainer.train()

/Users/dhanushkikkisetti/miniforge3/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3222
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2015
  Number of trainable parameters = 109483778


Step,Training Loss
500,0.130100
1000,0.056700
1500,0.049500
2000,0.040200


Saving model checkpoint to output/checkpoint-500
Configuration saved in output/checkpoint-500/config.json
Model weights saved in output/checkpoint-500/pytorch_model.bin
Saving model checkpoint to output/checkpoint-1000
Configuration saved in output/checkpoint-1000/config.json
Model weights saved in output/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to output/checkpoint-1500
Configuration saved in output/checkpoint-1500/config.json
Model weights saved in output/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to output/checkpoint-2000
Configuration saved in output/checkpoint-2000/config.json
Model weights saved in output/checkpoint-2000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=2015, training_loss=0.06898180125073226, metrics={'train_runtime': 19766.8138, 'train_samples_per_second': 0.815, 'train_steps_per_second': 0.102, 'total_flos': 4238719101849600.0, 'train_loss': 0.06898180125073226, 'epoch': 5.0})

In [18]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 806
  Batch size = 8


<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.03735582157969475,
 'eval_Accuracy Score ': 0.988833746898263,
 'eval_Recall Score ': 0.9826302729528535,
 'eval_Precission Score': 0.9949748743718593,
 'eval_F1 Score ': 0.9887640449438202,
 'eval_runtime': 160.7339,
 'eval_samples_per_second': 5.014,
 'eval_steps_per_second': 0.628,
 'epoch': 5.0}

In [8]:
source='/Users/dhanushkikkisetti/Documents/Research Assistant/right_left_finetunemodel'
#trainer.save_model(source)


In [9]:
#loaded_tokenizer=BertTokenizer.from_pretrained(source)
loaded_model=BertForSequenceClassification.from_pretrained(source)

In [10]:
x_test_token=tokenizer(x_test,padding=True,truncation=True,max_length=512,return_tensors='pt')
output=loaded_model(**x_test_token)


In [23]:
prediction_label=output.logits.detach().numpy()
prediction_label=np.argmax(prediction_label,axis=1)
accuracy=accuracy_score(y_pred=prediction_label,y_true=y_test)
accuracy

0.9876543209876543